In [1]:
from mlx_lm import load, generate
import pandas as pd

In [2]:
%%bash
python -m mlx_lm.convert --hf-path jphme/em_german_leo_mistral --mlx-path model/ -q 

[INFO] Loading


Fetching 8 files: 100%|██████████| 8/8 [00:00<00:00, 84096.32it/s]


[INFO] Quantizing


In [3]:
# TODO: Choose one
df = pd.read_csv("interpreted_data_mlx.csv")
#inter_df = pd.read_csv("test2.csv")

In [4]:
# TODO: Choose which row to start from
n=12386
inter_df = df.iloc[n:]

In [5]:
inter_df.head()

Unnamed: 0  patientid     sex   age  \
12386       12386    62714.0  female  44.0   
12387       12387    62715.0  female  49.0   
12388       12388    62717.0  female  78.0   
12389       12389    62719.0  female  27.0   
12390       12390    62720.0    male  52.0   

                                                     ANA  \
12386                     Beruf: Hausfrau; Sport: Joggen   
12387         Pat stellt sich mit aktuellen Bildern vor.   
12388                   Beruf: Rentner; Sport: Gymnastik   
12389  6 Wochen nach Skiunfall bisher Vollbelastug mi...   
12390  Pat stellt sich heute erneut zur weiteren Plan...   

                                                     EXA  \
12386  li Knie: keine Rötung/Überwärmung; klin.kein G...   
12387  Beinachse Genu varum 1 QF; flüss gangbild li k...   
12388  li Knie: keine Rötung/Überwärmung; 0-0-130;  p...   
12389  Beinachse: 2 QF genum varum re.Knie: keine Röt...   
12390  Beinachse:  2-3 QF genum varum li.Knie: keine ...   

                                                DIA_text  DIA_code  \
12386                  Z.n.Meniskusteilresektion li 1992  M23.3- Z   
12387  Sonstige Meniskusschädigungen: sonstiger und n...  M23.33 V   
12388  Sonstige Meniskusschädigungen: sonstiger und n...  M23.36 G   
12389  Verstauchung und Zerrung sonstiger und nicht n...   S83.6 V   
12390  Sonstige Meniskusschädigungen: Hinterhorn des ...  M23.32 G   

                                                 OP_text   OP_code  \
12386                                                NaN       NaN   
12387  Arthroskopische Operation am Gelenkknorpel und...  5-812.fh   
12388                                                NaN       NaN   
12389                                                NaN       NaN   
12390  Arthroskopische Operation am Gelenkknorpel und...   5-812.5   

                                           EXA_formatted EXA_interpreted  
12386  links knie keine rötung/überwärmung, kein gele...             NaN  
12387  flüss gangbild links knie keine rötung/überwär...             NaN  
12388  links knie keine rötung/überwärmung, keine übe...             NaN  
12389  beinachse 2 qf genum varum reknie keine rötung...             NaN  
12390  beinachse  2-3 qf genum varum liknie keine röt...             NaN

In [6]:
model, tokenizer = load("model/")

In [7]:
few_shot_examples = [
    ("links knie diskr schwellung, kein anteromedialer druckschmerz med äußerechts Seite des Kniegelenks, kein druckschmerz kniescheiben, keine meniskuszeichen rechts knie keine schwellung, druckschmerz proximal seitlich kniescheibe", 
     "Mein linkes Knie hat keine Schwellung, und es gibt keine Schmerzen beim Drücken auf der äußerechts Seite des Kniegelenks und der Kniescheibe. Mein rechtes Knie hat auch keine Schwellung, aber in der Nähe der Seite der Kniescheibe, gibt es Schmerzen beim Drücken."),
    ("rechts knie deutlicher schwellung, keine überstreckung des knies, streckdefizit des knies, angemessene beugung des knies, keine rötung", 
     "Mein rechtes Knie hat eine Schwellung, ich kann es nicht überstrecken und auch nicht komplett strecken. Die Beugung des Knies ist nur angemessen möglich. Mein Knie ist nicht gerötet."),
]

# Construct the few-shot prompt
instruction = "Du bist ein Patient mit den folgenden Symptomen. Ersetze alle lateinischen und deutschen Fachbegriffe und formuliere es umgangssprachlich aus deiner Sicht."
few_shot_prompt = f"{instruction}\n\n" + "\n\n".join([f"USER: {inp}\nASSISTANT: {out}" for inp, out in few_shot_examples])

In [8]:
def generate_response(text):
    if (text != "") and (text is not None) and not (isinstance(text, float)):
        prompt = f"{few_shot_prompt}\n\nUSER: {text}\nASSISTANT:"
        generated_text = generate(model, tokenizer, prompt=prompt, max_tokens=200, verbose=False)
        return generated_text
    return ""

In [9]:
def remove_after_newline(text):
    return text.split('\n')[0] if isinstance(text, str) else text

In [10]:
for idx, row in inter_df.iterrows():
    text = row['EXA_formatted']
    ouput_text = generate_response(text)
    inter_df.at[idx, 'EXA_interpreted'] = ouput_text

In [11]:
inter_df["EXA_interpreted"] = inter_df["EXA_interpreted"].apply(remove_after_newline)

/var/folders/4_/5vc_xrv55mjf8lnnrnlx1vz40000gn/T/ipykernel_4039/1584057196.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inter_df["EXA_interpreted"] = inter_df["EXA_interpreted"].apply(remove_after_newline)


In [12]:
inter_df.to_csv("interpreted_data_mlx_part2.csv")

In [13]:
# TODO: Use this value when rerunning notebook!
n = inter_df.EXA_interpreted.count()

In [14]:
n = 12386